In [0]:
df_squier = spark.read.format("csv")\
    .option("header","true")\
    .option("mode","FAILFAST")\
    .option("inferSchema","true")\
    .load("/FileStore/tables/guitar_squier.csv")

df_yamaha = spark.read.format("csv")\
    .option("header","true")\
    .option("mode","FAILFAST")\
    .option("inferSchema","true")\
    .load("/FileStore/tables/guitar_yamaha.csv")

df_epiphone = spark.read.format("csv")\
    .option("header","true")\
    .option("mode","FAILFAST")\
    .option("inferSchema","true")\
    .load("/FileStore/tables/guitar_epiphone.csv")

df_cort = spark.read.format("csv")\
    .option("header","true")\
    .option("mode","FAILFAST")\
    .option("inferSchema","true")\
    .load("/FileStore/tables/guitar_cort.csv")


# Limpiando los datos

Primero procesamos cada data frame por separado para poder atender a los problemas particulares de cada uno.

Cómo los titulos de las publicaciones extraidas tienen muchos datos, la idea es reducirlo a estos 4 campos:

- Marca (Cort, Squier, Epiphone, Yamaha)
- Tipo (Eléctrica, Acústica, Criolla)
- Serie 
- Modelo

Asi tener 4 columnas que correspondan a esos 4 campos y buscar llenarlas con las coincidencias que encuentre en el titulo. Y por otro lado tener la columna con el precio.

In [0]:
from pyspark.sql.functions import col, when, lower, udf, regexp_replace, round, count, lit,avg, substring
from pyspark.sql.types import StringType, ArrayType


In [0]:
# Acá esta la lógica para el match de palabras clave
# 'extract_from_list_udf_creator' para crear la UDF correspondiente a cada lista de la palabras en particular
# y la segunda 'extract_from_list' contiene la lógica para extraer la palabra coincidente en la lista, o en caso contrario, devolver null

def extract_from_list_udf_creator(word_list):
    def extract_from_list(text):
        for word in word_list:
            if word in text:
                return word
        return None
    return udf(extract_from_list, StringType())

## 1. Guitarras marca 'Cort'

In [0]:
guitar_type = ["criolla", "acustica", "eléctrica", "clasica", "clásica", "electrica", "acústica", "concierto", "classic", "electric", "acustic", "electrocriolla", "electroacústica", "electroacustica"]

brand = ["cort"]

cort_model = [
    "g100", "g200", "g250", 
    "m200", "m600", "m800", 
    "x100", "x11", "x500","x280","x300" 
    "earth 100", "earth 200", "earth 300", 
    "grand regal 100", "grand regal 200"
]

cort_series = [
    " g "," x ", 
    "earth","grand regal"
]

extract_type_udf = extract_from_list_udf_creator(guitar_type)
extract_brand_udf = extract_from_list_udf_creator(brand)
extract_model_udf = extract_from_list_udf_creator(cort_model)
extract_series_udf = extract_from_list_udf_creator(cort_series)

df_cort = df_cort.withColumn("title_lower", lower(col("title"))) \
    .withColumn("type", extract_type_udf(col("title_lower"))) \
    .withColumn("brand", extract_brand_udf(col("title_lower"))) \
    .withColumn("model", extract_model_udf(col("title_lower"))) \
    .withColumn("series", extract_series_udf(col("title_lower"))) 

### Observo la cantidad de valores nulos

In [0]:
df_cort.select('title_lower','type','brand','model','series').show(200)

+--------------------+---------+-----+-----+------+
|         title_lower|     type|brand|model|series|
+--------------------+---------+-----+-----+------+
|guitarra eléctric...|eléctrica| cort| x100|    x |
|guitarra acústica...| acústica| cort| null|  null|
|guitarra eléctric...|eléctrica| cort| null|  null|
|guitarra stratoca...|     null| cort| null|  null|
|guitarra eléctric...|eléctrica| cort| x100|    x |
|guitarra acústica...| acústica| cort| null|  null|
|guitarra eléctric...|eléctrica| cort| null|    g |
|guitarra cort str...|     null| cort| null|  null|
|guitarra acústica...| acústica| cort| null|  null|
|guitarra eléctric...|eléctrica| cort| x100|    x |
|guitarra cort x70...|     null| cort| null|  null|
|guitarra eléctric...|eléctrica| null| null|  null|
|guitarra eléctric...|eléctrica| cort| null|  null|
|guitarra electroa...| acústica| cort| null|  null|
|guitarra eléctric...|eléctrica| cort| null|  null|
|guitarra eléctric...|eléctrica| cort| x100|  null|
|guitarra el

In [0]:
nulos_tipo = df_cort.filter(col("type").isNull()).count()

nulos_serie = df_cort.filter(col("series").isNull()).count()

nulos_modelo = df_cort.filter(col("model").isNull()).count()

print(f"null types: {nulos_tipo}")
print(f"null series: {nulos_serie}")
print(f"null models: {nulos_modelo}")
print(f"de {df_cort.count()} resultados")

null types: 46
null series: 513
null models: 552
de 608 resultados


### Problema 1: Demasiados valores nulos

Muchos modelos y series no se seleccionaron porque no estan mi lista de modelos y series, lo que deja la tabla con demasiados valores nulos. Pero observando la columna 'title_lower' me doy cuenta que los modelos y series de estas guitarras tienen un patrón que puedo buscar:

- una letra y 3 numeros
- dos letras y 3 numeros

Algo así como: g300 o kr120

Por lo que la solución es buscar ese patrón en 'title_lower' para poder completar los modelos y series que faltan.

In [0]:
from pyspark.sql.functions import regexp_extract, col,avg

# creo la expresión regular para encontrar el patrón 'una letra y 3 números' o 'dos letras y 3 números'
pattern = r"([a-zA-Z]\d{3}|[a-zA-Z]{2}\d{3})"

# Busco los matches
df_cort_update = df_cort.withColumn("match", regexp_extract(col("title_lower"), pattern, 0))

# Observo si funciono
df_cort_update.select('title_lower','match').show(200)


+--------------------+-----+
|         title_lower|match|
+--------------------+-----+
|guitarra eléctric...| x100|
|guitarra acústica...|af515|
|guitarra eléctric...|kx507|
|guitarra stratoca...| g110|
|guitarra eléctric...| x100|
|guitarra acústica...|af510|
|guitarra eléctric...| g260|
|guitarra cort str...| g110|
|guitarra acústica...|ad810|
|guitarra eléctric...| x100|
|guitarra cort x70...| x700|
|guitarra eléctric...|le362|
|guitarra eléctric...| g110|
|guitarra electroa...|ad880|
|guitarra eléctric...|cr200|
|guitarra eléctric...|kx100|
|guitarra electroa...|ad880|
|guitarra eléctric...|kx100|
|guitarra eléctric...|kx307|
|guitarra acustica...|af510|
|guitarra acústica...|ad810|
|guitarra electric...| g110|
|guitarra eléctric...|cr100|
|guitarra electric...| g110|
|guitarra electroa...|af510|
|guitarra electric...| g200|
|guitarra acustica...|ad810|
|guitarra acústica...|af510|
|guitarra eléctric...|cr250|
|guitarra super st...|     |
|guitarra stratoca...| g110|
|guitarra acús

In [0]:
# ahora que tenemos los matches correspondientes, tenemos que separar la serie (las letras) del match para completar la columna series
# en la columna 'model' corresponde todo el match.

df_cort_update = df_cort_update.withColumn(
    "model", 
    when(col("match") != "", col("match")).otherwise(col("model"))
)

# extraemos solo las letras
df_cort_update = df_cort_update.withColumn(
    "series",
    when(col("match") != "", regexp_replace(col("match"), "[^a-zA-Z]", ""))
    .otherwise(col("series"))
)

# Observo el resultado
df_cort_update.select("title_lower", "match", "model", "series").show(200, False)


+--------------------------------------------------------------------------------------------------------------------------------------+-----+-----+------+
|title_lower                                                                                                                           |match|model|series|
+--------------------------------------------------------------------------------------------------------------------------------------+-----+-----+------+
|guitarra eléctrica cort x series x100 de meranti black poro abierto con diapasón de jatoba                                            |x100 |x100 |x     |
|guitarra acústica cort standard af515ce para diestros natural open pore                                                               |af515|af515|af    |
|guitarra eléctrica cort kx507 de caoba star dust green con diapasón de ébano                                                          |kx507|kx507|kx    |
|guitarra stratocaster cort g110 opsb open pore sunburst        

In [0]:
# y ahora vemos como logramos reducir la cantidad de nulos

nulos_type = df_cort_update.filter(col("type").isNull()).count()

nulos_series = df_cort_update.filter(col("series").isNull()).count()

nulos_model = df_cort_update.filter(col("model").isNull()).count()

print(f"null types: {nulos_type}")
print(f"null series: {nulos_series}")
print(f"null models: {nulos_model}")
print(f"de {df_cort.count()} resultados")

null types: 46
null series: 181
null models: 254
de 608 resultados


### Problema 2: Nulos en la columna 'Type'

El único caso donde la columna 'type' puede ser null es un caso donde también sea null la columna 'series' ya que si conocemos la serie, conocemos si es 'eléctrica' o 'acústica'.

No estoy seguro de cual serie corresponde a que tipo. Por eso sobre los datos que ya tengo intento averiguarlo agrupando tipos de guitarra por series. Luego a partir de esa información colocar el tipo corresponiente según la serie de la guitarra.

In [0]:
resultados = (
    df_cort_update
    .groupBy("series", "type")
    .agg(count("*").alias("amount"))
    .orderBy("series", "type") 
)

resultados.show(200, False)

+-----------+---------+------+
|series     |type     |amount|
+-----------+---------+------+
|null       |null     |8     |
|null       |acustica |42    |
|null       |acústica |51    |
|null       |clasica  |3     |
|null       |clásica  |20    |
|null       |criolla  |25    |
|null       |electrica|9     |
|null       |eléctrica|23    |
| g         |eléctrica|2     |
|ac         |acústica |1     |
|ac         |clasica  |5     |
|ac         |clásica  |11    |
|ac         |criolla  |14    |
|ad         |acustica |35    |
|ad         |acústica |45    |
|af         |acustica |20    |
|af         |acústica |36    |
|af         |eléctrica|1     |
|ap         |null     |1     |
|cr         |null     |1     |
|cr         |electrica|2     |
|cr         |eléctrica|10    |
|earth      |null     |1     |
|earth      |acustica |23    |
|earth      |acústica |46    |
|g          |null     |14    |
|g          |electrica|12    |
|g          |eléctrica|43    |
|grand regal|acústica |1     |
|kx     

In [0]:
# puedo notar también que para la serie g la tengo escrita como 'g' y ' g'. Lo arreglo con lo siguiente:
df_cort_update = df_cort_update.withColumn(
    "series",
    when(col('series').isin(' g '),'g').otherwise(col("series")) 
)

Podemos ver que las series y su tipo correspondiente es: 

- eléctrica: g, kr, kx, x, cr
- acústica earth

solamente me fijo en donde para esa serie existen tipos que son null, siendo asi que no existen types nulos para esa serie.

In [0]:
df_cort_update = df_cort_update.withColumn(
    "type",
    when(col("series").isin("x", "kr", "g","kx","cr"), "electrica")
    .when(col("series").isin("earth","af"), "acustica")
    .otherwise(col("type")) 
)

In [0]:
# Veo la diferencia
nulos_type = df_cort_update.filter(col("type").isNull()).count()
print(f"null types: {nulos_type}")
print(f"De {df_cort_update.count()} resultados")

null types: 9
De 608 resultados


### Filtramos los nulos restantes

In [0]:
# Veo cuantos casos con null me quedan en 'type' o 'series' para asegurarme de no estar perdiendo muchos datos. Considero que no saber el modelo es aceptable, pero no saber ni el tipo o la serie es prácticamente no saber nada de la guitarra.

df_cort_update.filter(col("type").isNull() | col("series").isNull()).select('title_lower','model','series','type').count()

Out[14]: 182

In [0]:
# Filtro estas ultimas filas donde no sabemos ni el tipo o la serie.
df_cort_clean = df_cort_update.filter(~(col("series").isNull() | col("type").isNull()))\
    .select('brand','type','series','model','price','title_lower')

### Problema 3: más de 2 o 3 palabras para decir lo mismo en 'type'

Cómo en las publicaciones algunas personas escriben 'acústica' con acento y otras 'acustica' sin acento. Asi como 'Criolla' y 'Clasica' que hacen referencia a lo mismo. Tenemos varias palabras para decir lo mismo por lo que la solución que se me ocurre es:
1. Ver cuales son esas palabras
2. Cambiarlas todas por su palabra en ingles (electric, acustic, classic)

In [0]:
df_cort_clean.groupBy('type').count().show()

#primero busco bien las palabras que se repiten para decir lo mismo 

+---------+-----+
|     type|count|
+---------+-----+
|eléctrica|    1|
|  criolla|   14|
| acustica|  174|
|electrica|  160|
| acústica|   61|
|  clásica|   11|
|  clasica|    5|
+---------+-----+



In [0]:
# modifico las coincidencias

df_cort_clean = df_cort_clean.withColumn(
    "type",
    when(col('type').isin("eléctrica","electrica"),"electric")
    .when(col('type').isin("criolla","clásica","clasica"),"classic")
    .when(col('type').isin("acústica","acustica"),"acustic")
)

### Problema 4: Nulos en 'brand'

In [0]:
# corroboro si me quedo alguna fila con 'brand' en null
df_cort_clean.filter(col("brand").isNull()).show()

+-----+--------+------+-----+-------+--------------------+
|brand|    type|series|model|  price|         title_lower|
+-----+--------+------+-----+-------+--------------------+
| null|electric|    le|le362|255.873|guitarra eléctric...|
+-----+--------+------+-----+-------+--------------------+



In [0]:
# la elimino, ya que no tengo seguridad que pertenezca a cort
df_cort_clean = df_cort_clean.filter(col("brand").isNotNull())

### Problema 5: Cambiar el tipo de la columna de price de string a float

In [0]:
#Transformamos la columna precio en una columna de tipo flotante, primero quitando el punto del string.
df_cort_clean = df_cort_clean.withColumn("price", regexp_replace(col("price"), "\\.", "").cast("float"))


### Data Frame final

In [0]:
# Observo el data frame ahora ya procesado y cuantas filas me quedaron
print(f"de: {df_cort.count()} quedaron: {df_cort_clean.count()}")
df_cort_final = df_cort_clean.select("brand","type","series","model","price")

df_cort_final.show(df_cort_clean.count(),False)

de: 608 quedaron: 425
+-----+--------+-----------+-----+---------+
|brand|type    |series     |model|price    |
+-----+--------+-----------+-----+---------+
|cort |electric|x          |x100 |467727.0 |
|cort |acustic |af         |af515|356318.0 |
|cort |electric|kx         |kx507|1580162.0|
|cort |electric|g          |g110 |449350.0 |
|cort |electric|x          |x100 |467727.0 |
|cort |acustic |af         |af510|349700.0 |
|cort |electric|g          |g260 |1099793.0|
|cort |electric|g          |g110 |442445.0 |
|cort |acustic |ad         |ad810|381995.0 |
|cort |electric|x          |x100 |467727.0 |
|cort |electric|x          |x700 |2654673.0|
|cort |electric|g          |g110 |345664.0 |
|cort |acustic |ad         |ad880|614810.0 |
|cort |electric|cr         |cr200|862140.0 |
|cort |electric|kx         |kx100|474048.0 |
|cort |acustic |ad         |ad880|543181.0 |
|cort |electric|kx         |kx100|556840.0 |
|cort |electric|kx         |kx307|1036586.0|
|cort |acustic |af         |af510

## 2. Guitarras marca 'Squier'

En el caso de las squier pasa lo mismo puede aceptarse que falte el modelo, pero no la serie, el tipo o la marca.

In [0]:
# observo la cantidad y el data frame por arriba 
squier_count = df_squier.count()
print(squier_count)
df_squier.show(squier_count,False)

221
+-----------------------------------------------------------------------------------------------------------------------------------------------+---------+
|title                                                                                                                                          |price    |
+-----------------------------------------------------------------------------------------------------------------------------------------------+---------+
|Guitarra Eléctrica Fender Squier Bullet Telecaster Black Mástil Laurel Indio                                                                   |521.039  |
|Guitarra eléctrica Squier by Fender Bullet Stratocaster HT de álamo black brillante con diapasón de laurel indio                               |453.077  |
|Guitarra eléctrica Squier by Fender Bullet Stratocaster HT de álamo brown sunburst brillante con diapasón de laurel indio                      |453.077  |
|Guitarra eléctrica Squier by Fender Affinity Series Stratoc

In [0]:
# declaro las listas con las palabras clave
guitar_type = ["eléctrica","electrica"]

brand = ["squier"]

squier_model = [
    "telecaster", "stratocaster", "jazzmaster", "jaguar hh"
]

squier_series = [
    "affinity", "bullet", "classic vibe", "contemporary", "contemporary jaguar hh"
]

# creo las UDF
extract_type_udf = extract_from_list_udf_creator(guitar_type)
extract_brand_udf = extract_from_list_udf_creator(brand)
extract_model_udf = extract_from_list_udf_creator(squier_model)
extract_series_udf = extract_from_list_udf_creator(squier_series)

# hago las transformaciones
df_squier = df_squier.withColumn("title_lower", lower(col("title"))) \
    .withColumn("type", extract_type_udf(col("title_lower"))) \
    .withColumn("brand", extract_brand_udf(col("title_lower"))) \
    .withColumn("model", extract_model_udf(col("title_lower"))) \
    .withColumn("series", extract_series_udf(col("title_lower"))) 

### Observo los valores nulos

In [0]:
df_squier.select('title_lower','brand','type','series','model').show(300,False)

+-----------------------------------------------------------------------------------------------------------------------------------------------+------+---------+------------+------------+
|title_lower                                                                                                                                    |brand |type     |series      |model       |
+-----------------------------------------------------------------------------------------------------------------------------------------------+------+---------+------------+------------+
|guitarra eléctrica fender squier bullet telecaster black mástil laurel indio                                                                   |squier|eléctrica|bullet      |telecaster  |
|guitarra eléctrica squier by fender bullet stratocaster ht de álamo black brillante con diapasón de laurel indio                               |squier|eléctrica|bullet      |stratocaster|
|guitarra eléctrica squier by fender bullet stratocaste

In [0]:
nulos_brand = df_squier.filter(col("brand").isNull()).count()

nulos_tipo = df_squier.filter(col("type").isNull()).count()

nulos_serie = df_squier.filter(col("series").isNull()).count()

nulos_modelo = df_squier.filter(col("model").isNull()).count()

print(f"null brand: {nulos_brand}")
print(f"null types: {nulos_tipo}")
print(f"null series: {nulos_serie}")
print(f"null models: {nulos_modelo}")
print(f"resultados sin nulos en type o series: {df_squier.filter(col('type').isNotNull() & col('series').isNotNull()).count()}")
print(f"de {df_squier.count()} resultados")

null brand: 20
null types: 42
null series: 88
null models: 27
resultados sin nulos en type o series: 105
de 221 resultados


### Problema 1: Demasiados valore nulos en 'series' y 'types'

In [0]:
# los casos donde la serie no es null pero si el type podemos resolverlos ya que sabemos que van a ser guitarras eléctricas
# veamos si encontramos casos asi

df_squier.filter(col('type').isNull() & col('series').isNotNull()).select('title_lower','type','brand','series').show(300,False)

+---------------------------------------------------------------------------------------------------------------------------+----+------+------------+
|title_lower                                                                                                                |type|brand |series      |
+---------------------------------------------------------------------------------------------------------------------------+----+------+------------+
|guitarra squier bullet stratocaster hardtail la plata color negro orientación de la mano diestro                           |null|squier|bullet      |
|squier affinity stratocaster hh guitarra doble humbucker                                                                   |null|squier|affinity    |
|pack guitarra squier telecaster bullet bk ampli accesorios                                                                 |null|squier|bullet      |
|combo guitarra squier bullet telecaster negra + amplificador                                 

In [0]:
df_squier = df_squier.withColumn(
    'type',
    when(col('series').isNotNull() & col('type').isNull(),'electric').otherwise(col('type'))
)

In [0]:
# vemos la diferencia
nulos_tipo = df_squier.filter(col("type").isNull()).count()
print(f"null types: {nulos_tipo}")
print(f"resultados sin nulos en type o series: {df_squier.filter(col('type').isNotNull() & col('series').isNotNull()).count()}")
print(f"de {df_squier.count()} resultados")

null types: 14
resultados sin nulos en type o series: 133
de 221 resultados


In [0]:
df_squier.show(300)

+--------------------+---------+--------------------+---------+------+------------+------------+
|               title|    price|         title_lower|     type| brand|       model|      series|
+--------------------+---------+--------------------+---------+------+------------+------------+
|Guitarra Eléctric...|  521.039|guitarra eléctric...|eléctrica|squier|  telecaster|      bullet|
|Guitarra eléctric...|  453.077|guitarra eléctric...|eléctrica|squier|stratocaster|      bullet|
|Guitarra eléctric...|  453.077|guitarra eléctric...|eléctrica|squier|stratocaster|      bullet|
|Guitarra eléctric...|  786.028|guitarra eléctric...|eléctrica|squier|stratocaster|    affinity|
|Guitarra eléctric...|  521.039|guitarra eléctric...|eléctrica|squier|  telecaster|      bullet|
|Guitarra eléctric...|  184.825|guitarra eléctric...|eléctrica|squier|  telecaster|contemporary|
|Guitarra eléctric...|  786.028|guitarra eléctric...|eléctrica|squier|stratocaster|    affinity|
|Guitarra eléctric...|  786.02

### Problemas que solucionamos en el Data Frame de Cort
- Mismas formas de referirse al tipo de guitarra
- Pasar el tipo de string a float en la columna price

In [0]:
# observo que en los casos nulos que nos quedan en la publicacion no corresponde con lo buscado, por lo que:
# eliminamos estas filas con null
# convertimos el precio a una columna de tipo float 
# pasamos los casos de 'electrica' o 'eléctrica' a electric

df_squier_clean = df_squier.filter(col('brand').isNotNull() & col('series').isNotNull() & col('type').isNotNull())

print(df_squier_clean.count())
df_squier_clean.show(200)

131
+--------------------+---------+--------------------+---------+------+------------+------------+
|               title|    price|         title_lower|     type| brand|       model|      series|
+--------------------+---------+--------------------+---------+------+------------+------------+
|Guitarra Eléctric...|  521.039|guitarra eléctric...|eléctrica|squier|  telecaster|      bullet|
|Guitarra eléctric...|  453.077|guitarra eléctric...|eléctrica|squier|stratocaster|      bullet|
|Guitarra eléctric...|  453.077|guitarra eléctric...|eléctrica|squier|stratocaster|      bullet|
|Guitarra eléctric...|  786.028|guitarra eléctric...|eléctrica|squier|stratocaster|    affinity|
|Guitarra eléctric...|  521.039|guitarra eléctric...|eléctrica|squier|  telecaster|      bullet|
|Guitarra eléctric...|  184.825|guitarra eléctric...|eléctrica|squier|  telecaster|contemporary|
|Guitarra eléctric...|  786.028|guitarra eléctric...|eléctrica|squier|stratocaster|    affinity|
|Guitarra eléctric...|  78

In [0]:
# en este caso sabemos que todas las guitarras son eléctricas
df_squier_clean = df_squier_clean.withColumn(
    'type',lit('electric')
)

In [0]:
#Transformamos la columna precio en una columna de tipo flotante, primero quitando el punto del string.

df_squier_clean = df_squier_clean.withColumn("price", regexp_replace(col("price"), "\\.", "").cast("float"))

### Data Frame final

In [0]:
# Observo el data frame ahora ya procesado y cuantas filas me quedaron
print(f"de: {df_squier.count()} quedaron: {df_squier_clean.count()}")
df_squier_final = df_squier_clean.select("brand","type","series","model","price")
df_squier_final.show(df_squier_clean.count(),False)

de: 221 quedaron: 131
+------+--------+------------+------------+---------+
|brand |type    |series      |model       |price    |
+------+--------+------------+------------+---------+
|squier|electric|bullet      |telecaster  |521039.0 |
|squier|electric|bullet      |stratocaster|453077.0 |
|squier|electric|bullet      |stratocaster|453077.0 |
|squier|electric|affinity    |stratocaster|786028.0 |
|squier|electric|bullet      |telecaster  |521039.0 |
|squier|electric|contemporary|telecaster  |184825.0 |
|squier|electric|affinity    |stratocaster|786028.0 |
|squier|electric|affinity    |stratocaster|786028.0 |
|squier|electric|bullet      |stratocaster|453639.0 |
|squier|electric|affinity    |telecaster  |829264.0 |
|squier|electric|affinity    |telecaster  |598357.0 |
|squier|electric|affinity    |stratocaster|683506.0 |
|squier|electric|affinity    |stratocaster|598357.0 |
|squier|electric|affinity    |stratocaster|683502.0 |
|squier|electric|affinity    |telecaster  |523238.0 |
|squie

## 3. Guitarras marca 'Epiphone'

In [0]:
# observo la cantidad y el data frame por arriba 
epi_count = df_epiphone.count()
print(epi_count)
df_epiphone.show(epi_count,False)

524
+--------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|title                                                                                                                                                   |price    |
+--------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|Guitarra eléctrica Epiphone Les Paul Special VE de álamo vintage sunburst con diapasón de palo de rosa                                                  |474.048  |
|Guitarra eléctrica Epiphone Slash “AFD” Les Paul Special II Outfit les paul special-ii de okoume appetite amber con diapasón de palo de rosa            |122.590  |
|Guitarra eléctrica Epiphone Original Collection Les Paul Standard 60s de caoba iced tea níquel con diapasón de laurel indio                             |2.199.586|
|Guita

In [0]:
# declaro las listas con las palabras clave
guitar_type = ["eléctrica","electrica"]

brand = ["epiphone"]

epi_model = [
    "les paul", "sg", "sheraton II pro"
]

epi_series = [
    " special I/II ", "studio", "standard", "custom", "modern", "melody maker",
    "special", "sheraton","es-335"
]

# creo las UDF
extract_type_udf = extract_from_list_udf_creator(guitar_type)
extract_brand_udf = extract_from_list_udf_creator(brand)
extract_model_udf = extract_from_list_udf_creator(epi_model)
extract_series_udf = extract_from_list_udf_creator(epi_series)

# hago las transformaciones
df_epiphone = df_epiphone.withColumn("title_lower", lower(col("title"))) \
    .withColumn("type", extract_type_udf(col("title_lower"))) \
    .withColumn("brand", extract_brand_udf(col("title_lower"))) \
    .withColumn("model", extract_model_udf(col("title_lower"))) \
    .withColumn("series", extract_series_udf(col("title_lower"))) 

### Observo la cantidad de valores nulos

In [0]:
df_epiphone.select('brand','type','series','model').show(500,False)

+--------+---------+------------+--------+
|brand   |type     |series      |model   |
+--------+---------+------------+--------+
|epiphone|eléctrica|special     |les paul|
|epiphone|eléctrica|special     |les paul|
|epiphone|eléctrica|standard    |les paul|
|epiphone|eléctrica|melody maker|les paul|
|epiphone|eléctrica|null        |sg      |
|epiphone|eléctrica|standard    |les paul|
|epiphone|eléctrica|special     |les paul|
|epiphone|eléctrica|studio      |les paul|
|epiphone|eléctrica|modern      |les paul|
|epiphone|eléctrica|studio      |les paul|
|epiphone|eléctrica|studio      |les paul|
|epiphone|eléctrica|melody maker|les paul|
|epiphone|electrica|special     |null    |
|epiphone|null     |special     |null    |
|epiphone|null     |custom      |null    |
|epiphone|eléctrica|modern      |sg      |
|epiphone|eléctrica|melody maker|les paul|
|epiphone|eléctrica|melody maker|les paul|
|epiphone|null     |null        |les paul|
|epiphone|null     |special     |les paul|
|epiphone|e

In [0]:
# contamos los datos nulos

nulos_brand = df_epiphone.filter(col("brand").isNull()).count()

nulos_tipo = df_epiphone.filter(col("type").isNull()).count()

nulos_serie = df_epiphone.filter(col("series").isNull()).count()

nulos_modelo = df_epiphone.filter(col("model").isNull()).count()

print(f"null brand: {nulos_brand}")
print(f"null types: {nulos_tipo}")
print(f"null series: {nulos_serie}")
print(f"null models: {nulos_modelo}")
print(f"resultados sin nulos en type o series: {df_epiphone.filter(col('type').isNotNull() & col('series').isNotNull()).count()}")
print(f"de {df_epiphone.count()} resultados")

null brand: 1
null types: 212
null series: 272
null models: 179
resultados sin nulos en type o series: 156
de 524 resultados


### Problema 1: Demasiados nulos en 'series'

In [0]:
# intento analizar los casos donde 'series' es null para ver si se puede hacer algo al respecto

df_epiphone.filter(col('series').isNull()).select('title_lower','series','model').show(350,False)

+---------------------------------------------------------------------------------------------------------------------------------------+------+--------+
|title_lower                                                                                                                            |series|model   |
+---------------------------------------------------------------------------------------------------------------------------------------+------+--------+
|guitarra eléctrica epiphone original collection casino worn de arce blue denim desgastado con diapasón de laurel indio                 |null  |sg      |
|combo guitarra epiphone les paul ve funda encordado gratis                                                                             |null  |les paul|
|kit guitarra epiphone les paul ve con funda encordado gratis                                                                           |null  |les paul|
|guitarra epiphone casino worn denim eléctrica hollow body                  

In [0]:
# No reconozco una forma de obtener los modelos que faltan o de identifiarlos entre los nombres, asi que solamente me voy a quedar con los que fueron claramente identificados

df_epiphone_clean = df_epiphone.filter(col('series').isNotNull())

# Observo el data frame resultante y veo que falta completar el tipo en algunas filas

df_epiphone_clean.select('type','series','model','price').show(df_epiphone_clean.count(),False)

+---------+------------+--------+---------+
|type     |series      |model   |price    |
+---------+------------+--------+---------+
|eléctrica|special     |les paul|474.048  |
|eléctrica|special     |les paul|122.590  |
|eléctrica|standard    |les paul|2.199.586|
|eléctrica|melody maker|les paul|431.582  |
|eléctrica|standard    |les paul|214.803  |
|eléctrica|special     |les paul|545.156  |
|eléctrica|studio      |les paul|808.043  |
|eléctrica|modern      |les paul|165.423  |
|eléctrica|studio      |les paul|632.065  |
|eléctrica|studio      |les paul|742.676  |
|eléctrica|melody maker|les paul|375.288  |
|electrica|special     |null    |772.305  |
|null     |special     |null    |671.569  |
|null     |custom      |null    |1.590.039|
|eléctrica|modern      |sg      |1.466.391|
|eléctrica|melody maker|les paul|480.368  |
|eléctrica|melody maker|les paul|375.288  |
|null     |special     |les paul|856.675  |
|eléctrica|special     |les paul|606.781  |
|eléctrica|special     |les paul

### Problemas que solucionamos en el Data Frame de Cort
- Mismas formas de referirse al tipo de guitarra
- Pasar el tipo de string a float en la columna price

In [0]:
# completo las filas de type con 'electric' porque se que son guitarras electricas y paso la columna 'price' a flotante

df_epiphone_clean = df_epiphone_clean.withColumn(
    'type',lit('electric')
)

df_epiphone_clean = df_epiphone_clean.withColumn("price", regexp_replace(col("price"), "\\.", "").cast("float"))

In [0]:
df_epiphone_clean.select('model','price').orderBy('price').show(10)

+--------+--------+
|   model|   price|
+--------+--------+
|les paul| 93116.0|
|les paul|122590.0|
|      sg|141967.0|
|      sg|143202.0|
|      sg|143202.0|
|      sg|143202.0|
|      sg|149374.0|
|les paul|154395.0|
|les paul|165423.0|
|les paul|165423.0|
+--------+--------+
only showing top 10 rows



### Buscamos nulos en brand y los eliminamos

In [0]:
df_epiphone_clean.filter(col("brand").isNull()).show()

+--------------------+---------+--------------------+--------+-----+--------+--------+
|               title|    price|         title_lower|    type|brand|   model|  series|
+--------------------+---------+--------------------+--------+-----+--------+--------+
|Guitarra Les Paul...|2882216.0|guitarra les paul...|electric| null|les paul|standard|
+--------------------+---------+--------------------+--------+-----+--------+--------+



In [0]:
df_epiphone_clean = df_epiphone_clean.filter(col("brand").isNotNull())

### Data Frame final

In [0]:
# Observo el data frame ahora ya procesado y cuantas filas me quedaron
print(f"de: {df_epiphone.count()} quedaron: {df_epiphone_clean.count()}")
df_epiphone_final = df_epiphone_clean.select("brand","type","series","model","price").orderBy('price')
df_epiphone_final.show(df_epiphone_clean.count(),False)

de: 524 quedaron: 251
+--------+--------+------------+--------+---------+
|brand   |type    |series      |model   |price    |
+--------+--------+------------+--------+---------+
|epiphone|electric|special     |les paul|93116.0  |
|epiphone|electric|special     |les paul|122590.0 |
|epiphone|electric|modern      |sg      |141967.0 |
|epiphone|electric|modern      |sg      |143202.0 |
|epiphone|electric|modern      |sg      |143202.0 |
|epiphone|electric|modern      |sg      |143202.0 |
|epiphone|electric|special     |sg      |149374.0 |
|epiphone|electric|modern      |les paul|154395.0 |
|epiphone|electric|modern      |les paul|165423.0 |
|epiphone|electric|modern      |les paul|165423.0 |
|epiphone|electric|standard    |sg      |165423.0 |
|epiphone|electric|custom      |null    |196684.0 |
|epiphone|electric|standard    |les paul|214803.0 |
|epiphone|electric|standard    |les paul|214803.0 |
|epiphone|electric|modern      |sg      |242726.0 |
|epiphone|electric|modern      |sg      |2

## 4. Guitarras marca 'Yamaha'

In [0]:
# observo la cantidad filas y los datos

print(df_yamaha.count())
df_yamaha.show(380)

379
+--------------------+---------+
|               title|    price|
+--------------------+---------+
|Guitarra criolla ...|  298.000|
|Guitarra eléctric...|  538.012|
|Guitarra Electroa...|  370.955|
|Yamaha C40 Guitar...|  234.797|
|Guitarra Eléctric...|1.110.833|
|Guitarra Electroa...|  620.831|
|Guitarra criolla ...|  329.352|
|Guitarra eléctric...|  642.015|
|Guitarra Acustica...|  577.862|
|Guitarra criolla ...|  318.044|
|Guitarra criolla ...|  251.265|
|Guitarra eléctric...|  625.965|
|Guitarra Clasica ...|  234.797|
|Guitarra Electroa...|  499.988|
|Guitarra Criolla ...|  260.000|
|Yamaha Pacifica 1...|1.110.833|
|Guitarra Criolla ...|  259.855|
|Guitarra acústica...|  589.326|
|Guitarra Clasica ...|  244.000|
|Guitarra Acústica...|  537.017|
|Yamaha Paquete De...|  771.775|
|Guitarra acústica...|  606.784|
|Guitarra criolla ...|  559.999|
|Guitarra Electroa...|1.128.589|
|Guitarra criolla ...|2.218.085|
|Guitarra eléctric...|  535.013|
|Guitarra Yamaha N...|2.655.962|
|Guita

In [0]:
# declaro las listas con las palabras clave
guitar_type = ["criolla", "acustica", "eléctrica", "clasica", "clásica", "electrica", "acústica", "concierto", "classic", "electric", "acustic", "electrocriolla", "electroacústica", "electroacustica"]

brand = ["yamaha"]

yamaha_model = [
    "fg800", "fg830", "fgx800c", 
    "fg5", "fs800", "fsx800c", 
    "apx600", "apx1200ii", "012", 
    "112v", "611hfm", "element", 
    "professional","c40","c80","c70","apx700ii","apx700",
    "apx-700","apx-600", "fx370","fx310","f370","f310"
]

yamaha_series = [
    "fg", "fs", "apx", "pacifica", "cpx",
    "revstar","pac","fx"
]

# creo las UDF
extract_type_udf = extract_from_list_udf_creator(guitar_type)
extract_brand_udf = extract_from_list_udf_creator(brand)
extract_model_udf = extract_from_list_udf_creator(yamaha_model)
extract_series_udf = extract_from_list_udf_creator(yamaha_series)

# hago las transformaciones
df_yamaha = df_yamaha.withColumn("title_lower", lower(col("title"))) \
    .withColumn("type", extract_type_udf(col("title_lower"))) \
    .withColumn("brand", extract_brand_udf(col("title_lower"))) \
    .withColumn("model", extract_model_udf(col("title_lower"))) \
    .withColumn("series", extract_series_udf(col("title_lower"))) 

### Observo la cantidad de valores nulos

In [0]:
df_yamaha.select(substring(col('title_lower'),1,50),'brand','type','series','model').show(380,False)

+--------------------------------------------------+------+---------+--------+------------+
|substring(title_lower, 1, 50)                     |brand |type     |series  |model       |
+--------------------------------------------------+------+---------+--------+------------+
|guitarra criolla clásica yamaha c40 para diestros |yamaha|criolla  |null    |c40         |
|guitarra eléctrica yamaha pac012/100 series 012 st|yamaha|eléctrica|pac     |012         |
|guitarra electroacústica yamaha cx40 para diestros|yamaha|acústica |null    |null        |
|yamaha c40 guitarra clasica criolla de estudio pro|yamaha|criolla  |null    |c40         |
|guitarra eléctrica yamaha pac112v superstrato - pl|yamaha|eléctrica|pac     |112v        |
|guitarra electroacústica yamaha fsx315c natural - |yamaha|acústica |fs      |null        |
|guitarra criolla clásica yamaha c70 para diestros |yamaha|criolla  |null    |c70         |
|guitarra eléctrica yamaha pac012/100 series 012 st|yamaha|eléctrica|pac     |01

In [0]:
nulos_brand = df_yamaha.filter(col("brand").isNull()).count()

nulos_tipo = df_yamaha.filter(col("type").isNull()).count()

nulos_serie = df_yamaha.filter(col("series").isNull()).count()

nulos_modelo = df_yamaha.filter(col("model").isNull()).count()

print(f"null brand: {nulos_brand}")
print(f"null types: {nulos_tipo}")
print(f"null series: {nulos_serie}")
print(f"null models: {nulos_modelo}")
print(f"resultados sin nulos en type o series: {df_yamaha.filter(col('type').isNotNull() & col('series').isNotNull()).count()}")
print(f"de {df_yamaha.count()} resultados")

null brand: 1
null types: 28
null series: 152
null models: 181
resultados sin nulos en type o series: 217
de 379 resultados


### Problema 1. Null en 'series' pero no en modelo
Hay casos donde el modelo C40 o C80 esta en la columna model, siendo de serie 'C', pero no esta este modelo, ya que con el método que usamos es complicado asilar la 'C'. 
Por lo que a esos modelos les vamos a agregar la serie manualmente.

In [0]:
df_yamaha = df_yamaha.withColumn('series',
                     when(col('model').isin("c40","c80","c70"),'c').otherwise(col('series')))

df_yamaha.show(300)

+--------------------+---------+--------------------+---------+------+------------+--------+
|               title|    price|         title_lower|     type| brand|       model|  series|
+--------------------+---------+--------------------+---------+------+------------+--------+
|Guitarra criolla ...|  298.000|guitarra criolla ...|  criolla|yamaha|         c40|       c|
|Guitarra eléctric...|  538.012|guitarra eléctric...|eléctrica|yamaha|         012|     pac|
|Guitarra Electroa...|  370.955|guitarra electroa...| acústica|yamaha|        null|    null|
|Yamaha C40 Guitar...|  234.797|yamaha c40 guitar...|  criolla|yamaha|         c40|       c|
|Guitarra Eléctric...|1.110.833|guitarra eléctric...|eléctrica|yamaha|        112v|     pac|
|Guitarra Electroa...|  620.831|guitarra electroa...| acústica|yamaha|        null|      fs|
|Guitarra criolla ...|  329.352|guitarra criolla ...|  criolla|yamaha|         c70|       c|
|Guitarra eléctric...|  642.015|guitarra eléctric...|eléctrica|yamaha|

In [0]:
# Quiero observar las series que obtuvimos a ver si hay nombres repetidos


df_yamaha.groupBy('series','model').count().orderBy('series').show(100)

+--------+------------+-----+
|  series|       model|count|
+--------+------------+-----+
|    null|        null|   69|
|    null|        f370|   21|
|    null|        f310|   29|
|     apx|        null|    6|
|     apx|     apx-700|    4|
|     apx|      apx600|   20|
|     apx|    apx700ii|    9|
|     apx|     apx-600|    3|
|       c|         c70|    5|
|       c|         c40|   22|
|       c|         c80|    8|
|     cpx|        null|    8|
|      fg|     fsx800c|    3|
|      fg|        null|   16|
|      fg|       fg800|    3|
|      fg|       fg830|    2|
|      fg|       fs800|    3|
|      fg|     fgx800c|    2|
|      fs|     fsx800c|    1|
|      fs|       fs800|    1|
|      fs|        null|   21|
|      fx|       fx310|    5|
|      fx|        null|    6|
|      fx|       fx370|   16|
|     pac|        null|    9|
|     pac|        f310|    2|
|     pac|         012|   13|
|     pac|        112v|    3|
|pacifica|        null|   21|
|pacifica|         012|    9|
|pacifica|

### Problema 2. Dos formas de decir lo mismo
observo los resultados y veo que la serie 'pacifica' tiene dos formas de escribirse 'pacifica' y 'pac' 

In [0]:
# modifico las series 'pac' por 'pacifica' y en el modelo quito los numeros 'pac'
# también las apx-600 / apx-700 en model, las deberia pasar a 'apx600' 'apx700'

df_yamaha = df_yamaha.withColumn(
    'series',
    when(col('series').contains('pac'),'pacifica').otherwise(col('series')),
)

df_yamaha = df_yamaha.withColumn(
    'model',
    when(col('model').contains('apx-600'),'apx600').otherwise(col('model'))
)
    
df_yamaha_2 = df_yamaha.withColumn(
    'model',
    when(col('model').contains('apx-700'),'apx700').otherwise(col('model'))
)


df_yamaha_2.groupBy('series','type').count().orderBy('series').show(100,False)

+--------+---------+-----+
|series  |type     |count|
+--------+---------+-----+
|null    |null     |18   |
|null    |criolla  |27   |
|null    |acustica |37   |
|null    |electrica|1    |
|null    |acústica |30   |
|null    |clásica  |6    |
|apx     |acústica |35   |
|apx     |eléctrica|1    |
|apx     |acustica |6    |
|c       |clásica  |5    |
|c       |acústica |1    |
|c       |criolla  |27   |
|c       |clasica  |2    |
|cpx     |acústica |8    |
|fg      |eléctrica|3    |
|fg      |acústica |26   |
|fs      |acústica |14   |
|fs      |null     |1    |
|fs      |acustica |8    |
|fx      |acústica |19   |
|fx      |acustica |8    |
|pacifica|null     |8    |
|pacifica|eléctrica|37   |
|pacifica|acústica |2    |
|pacifica|electrica|14   |
|revstar |null     |1    |
|revstar |eléctrica|21   |
|revstar |electrica|13   |
+--------+---------+-----+



### Problema 3. Datos Erroneos

Puedo observar que 'pacifica' a pesar de ser solo guitarras electricas, tiene algunas muy pocas con tipo 'acústica' o 'criolla'.

Lo mismo pasa con las 'fg' y las 'apx' solo hay acústicas. No eléctricas.


In [0]:
# podemos ver que si son datos erroneos, ya que no son guitarras de serie 'pacifica'.

df_yamaha_2 = df_yamaha_2.where(~((col('series').contains('pacifica')) & (col('type').isin('acústica', 'acustica', 'criolla'))) | col('series').isNull() | col('type').isNull())


In [0]:
df_yamaha_2.where(((col('series').contains('fg')) & (col('type').isin('electric','electrica','eléctrica')))).select('title','series').show(100,False)

+------------------------------------------------------------+------+
|title                                                       |series|
+------------------------------------------------------------+------+
|Guitarra Eléctrica Yamaha Rss20fg Revstar Std Flash Green   |fg    |
|Guitarra Eléctrica Yamaha Rss20fg Revstar Std Flash Green Cu|fg    |
|Guitarra Eléctrica Yamaha Rss20fg Revstar Std Hot Merlot Cuo|fg    |
+------------------------------------------------------------+------+



en este caso es más conveniente directamente cambiar la series a 'revstar'

In [0]:
fg_bool = (col('series').contains('fg')) & (col('type').isin('electric','electrica','eléctrica'))

df_yamaha3 = df_yamaha_2.withColumn('series',when(fg_bool,'revstar').otherwise(col('series')))

df_yamaha3.filter(col('series').contains('fg')).select('series','type').show(100,False)

+------+--------+
|series|type    |
+------+--------+
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
|fg    |acústica|
+------+--------+



In [0]:
# ahora veo cual es el caso de la apx
df_yamaha3.where(((col('series').contains('apx')) & (col('type').isin('electric','electrica','eléctrica')))).select('title','series','type').show(100,False)

+---------------------------------------------------------+------+---------+
|title                                                    |series|type     |
+---------------------------------------------------------+------+---------+
|Guitarra eléctrica Yamaha Apx700ii Natural Apx-700 Apx700|apx   |eléctrica|
+---------------------------------------------------------+------+---------+



In [0]:
# vemos que solamente esta mal el tipo asi que lo corregimos

df_yamaha3 = df_yamaha3.withColumn('type',when(col('series').contains('apx'),'acustic').otherwise(col('type')))

In [0]:
## CON ESTE CÓDIGO PUDE ENCONTRAR EN LA COLUMNA TITLE UN MODELO Y MARCA QUE ANTES ERA NULL, AL VERLO LO AGREGUE A LA LISTA DE MODELOS Y MARCAS.

df_yamaha3.filter(col('type').isNull() | col('series').isNull())\
    .select(
        regexp_extract(col('title_lower'),r"\bfx\w{3}",0).alias('match1'),
        regexp_extract(col('title_lower'),r"\bf\w{3}",0).alias('match2')
        ).filter((col('match1') != "")).groupBy('match1').count().show(100)


+------+-----+
|match1|count|
+------+-----+
+------+-----+



In [0]:
# vemos cuales son los casos con null en 'series' o 'type', pero no el modelo, a ver si podemos obtener información

df_yamaha3.filter((col('type').isNull() | col('series').isNull()) & col('model').isNotNull()).select(col('title_lower').substr(1,40),'series','type','model').show(300,False)

+----------------------------------------+-------+--------+-------+
|substring(title_lower, 1, 40)           |series |type    |model  |
+----------------------------------------+-------+--------+-------+
|guitarra acustica folk yamaha f370 tapa |null   |acustica|f370   |
|guitarra acústica yamaha f310tbs tabacco|null   |acústica|f310   |
|guitarra acústica yamaha f310p para dies|null   |acústica|f310   |
|guitarra acústica tipo folk yamaha f370 |null   |acústica|f370   |
|guitarra acustica folk yamaha f370 tapa |null   |acustica|f370   |
|guitarra acustica folk yamaha f370 tapa |null   |acustica|f370   |
|guitarra acustica folk yamaha f310 tapa |null   |acustica|f310   |
|guitarra folk yamaha f310 cherry color s|null   |null    |f310   |
|guitarra folk yamaha f310 cherry sunburs|null   |null    |f310   |
| guitarra acustica folk yamaha f370 tapa|null   |acustica|f370   |
|guitarra acústica yamaha f310 dreadnough|null   |acústica|f310   |
|guitarra acústica yamaha f310 dreadnough|null  

In [0]:
# Podemos observar que hay un monton de guitarras modelo f370 que son serie f y tipo acústica que podemos ajustar

df_yamaha_t = df_yamaha3.withColumn('series',
                      when(col('model').isin('f370','f310'),'f').otherwise(col('series'))
                      )

df_yamaha_4 = df_yamaha_t.withColumn('type',
                      when(col('model').isin('f370','f310'),'acustic').otherwise(col('type'))
                      )

df_yamaha_4.filter(col('model').isin('f370','f310')).select('type','series').show(100)

+-------+------+
|   type|series|
+-------+------+
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
|acustic|     f|
+-------+------+



In [0]:
# analizo las que tienen como 'type' null

df_yamaha_4.filter(col('type').isNull()).select('type','series','model').groupBy('series').count().show(100)

+--------+-----+
|  series|count|
+--------+-----+
|pacifica|    8|
| revstar|    1|
|    null|   13|
|      fs|    1|
+--------+-----+



In [0]:
# podemos ver que varias son 'pacifica' o 'revstar' es decir de tipo electrica, mientras que 'fs' es de tipo acústica

df_yamaha_t = df_yamaha_4.withColumn('type',when(col('series').isin('pacifica','revstar'),'electric').otherwise(col('type')))

df_yamaha_5 = df_yamaha_t.withColumn('type',when(col('series').isin('fs'),'acustic').otherwise(col('type')))

df_yamaha_5.filter(col('type').isNull()).select('type','series','model').groupBy('series').count().show(100)

+------+-----+
|series|count|
+------+-----+
|  null|   13|
+------+-----+



### Filtramos los nulos restantes

In [0]:


df_yamaha_t = df_yamaha_5.select('brand','type','series','model','price')
df_yamaha_clean = df_yamaha_t.filter(col('brand').isNotNull() & col('type').isNotNull() & col('series').isNotNull())

df_yamaha_clean.show(df_yamaha_clean.count())

+------+--------+--------+------------+---------+
| brand|    type|  series|       model|    price|
+------+--------+--------+------------+---------+
|yamaha| criolla|       c|         c40|  298.000|
|yamaha|electric|pacifica|         012|  538.012|
|yamaha| criolla|       c|         c40|  234.797|
|yamaha|electric|pacifica|        112v|1.110.833|
|yamaha| acustic|      fs|        null|  620.831|
|yamaha| criolla|       c|         c70|  329.352|
|yamaha|electric|pacifica|         012|  642.015|
|yamaha| acustic|       f|        f370|  577.862|
|yamaha| criolla|       c|         c40|  318.044|
|yamaha| criolla|       c|         c40|  251.265|
|yamaha|electric|pacifica|         012|  625.965|
|yamaha| criolla|       c|         c40|  234.797|
|yamaha| criolla|       c|         c40|  260.000|
|yamaha|electric|pacifica|        112v|1.110.833|
|yamaha| criolla|       c|         c40|  259.855|
|yamaha|acústica|      fg|       fs800|  589.326|
|yamaha| clasica|       c|         c40|  244.000|


### Problemas que solucionamos en el Data Frame de Cort
- Mismas formas de referirse al tipo de guitarra
- Pasar el tipo de string a float en la columna price

In [0]:
# busco las variaciones
df_yamaha_clean.groupBy('type').count().orderBy(col('type')).show()

+--------+-----+
|    type|count|
+--------+-----+
| acustic|  115|
|acustica|    8|
|acústica|   54|
| clasica|    2|
| clásica|    5|
| criolla|   27|
|electric|   97|
+--------+-----+



In [0]:
# modifico las variaciones

df_yamaha_clean = df_yamaha_clean.withColumn(
    "type",
    when(col('type').isin("criolla","clásica","clasica"),"classic")
    .when(col('type').isin("acústica","acustica"),"acustic")
    .otherwise(col('type'))
)

In [0]:
# paso el price de string a float
df_yamaha_clean = df_yamaha_clean.withColumn("price", regexp_replace(col("price"), "\\.", "").cast("float"))

### Data Frame Final

In [0]:
df_yamaha_final = df_yamaha_clean
df_yamaha_final.show(df_yamaha_final.count())

+------+--------+--------+------------+---------+
| brand|    type|  series|       model|    price|
+------+--------+--------+------------+---------+
|yamaha| classic|       c|         c40| 298000.0|
|yamaha|electric|pacifica|         012| 538012.0|
|yamaha| classic|       c|         c40| 234797.0|
|yamaha|electric|pacifica|        112v|1110833.0|
|yamaha| acustic|      fs|        null| 620831.0|
|yamaha| classic|       c|         c70| 329352.0|
|yamaha|electric|pacifica|         012| 642015.0|
|yamaha| acustic|       f|        f370| 577862.0|
|yamaha| classic|       c|         c40| 318044.0|
|yamaha| classic|       c|         c40| 251265.0|
|yamaha|electric|pacifica|         012| 625965.0|
|yamaha| classic|       c|         c40| 234797.0|
|yamaha| classic|       c|         c40| 260000.0|
|yamaha|electric|pacifica|        112v|1110833.0|
|yamaha| classic|       c|         c40| 259855.0|
|yamaha| acustic|      fg|       fs800| 589326.0|
|yamaha| classic|       c|         c40| 244000.0|


# Uno todos los Data Frames en uno

In [0]:
df_guitars = df_cort_final.union(df_squier_final)
df_guitars = df_epiphone_final.union(df_guitars)
df_guitars = df_yamaha_final.union(df_guitars)
print(f"Puede obtener los datos de: {df_guitars.count()} guitarras")
df_guitars.show(df_guitars.count(),False)

Puede obtener los datos de: 1115 guitarras
+--------+--------+------------+------------+---------+
|brand   |type    |series      |model       |price    |
+--------+--------+------------+------------+---------+
|yamaha  |classic |c           |c40         |298000.0 |
|yamaha  |electric|pacifica    |012         |538012.0 |
|yamaha  |classic |c           |c40         |234797.0 |
|yamaha  |electric|pacifica    |112v        |1110833.0|
|yamaha  |acustic |fs          |null        |620831.0 |
|yamaha  |classic |c           |c70         |329352.0 |
|yamaha  |electric|pacifica    |012         |642015.0 |
|yamaha  |acustic |f           |f370        |577862.0 |
|yamaha  |classic |c           |c40         |318044.0 |
|yamaha  |classic |c           |c40         |251265.0 |
|yamaha  |electric|pacifica    |012         |625965.0 |
|yamaha  |classic |c           |c40         |234797.0 |
|yamaha  |classic |c           |c40         |260000.0 |
|yamaha  |electric|pacifica    |112v        |1110833.0|
|yama

In [0]:
# descargo el data frame final a través de una consulta en SQL
df_guitars.createOrReplaceTempView("guitars_view")


In [0]:
%sql 

SELECT * FROM guitars_view

brand,type,series,model,price
yamaha,classic,c,c40,298000.0
yamaha,electric,pacifica,012,538012.0
yamaha,classic,c,c40,234797.0
yamaha,electric,pacifica,112v,1110833.0
yamaha,acustic,fs,null,620831.0
yamaha,classic,c,c70,329352.0
yamaha,electric,pacifica,012,642015.0
yamaha,acustic,f,f370,577862.0
yamaha,classic,c,c40,318044.0
yamaha,classic,c,c40,251265.0


# Paso de un esquema relacional a un esquema estrella


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
# Dimensiones
dim_brand = df_guitars.select("brand").distinct().withColumn("id_brand", monotonically_increasing_id())
dim_type = df_guitars.select("type").distinct().withColumn("id_type", monotonically_increasing_id())
dim_series = df_guitars.select("series").distinct().withColumn("id_series", monotonically_increasing_id())

# Agregar IDs a df
df = df_guitars.join(dim_brand, on="brand") \
       .join(dim_type, on="type") \
       .join(dim_series, on="series")

# Crear dim_modelo
dim_model = df.select("model", "id_brand", "id_type", "id_series").distinct() \
               .withColumn("id_model", monotonically_increasing_id())

# Tabla de hechos
fact_guitar = df.join(dim_model, on=["model", "id_brand", "id_type", "id_series"]) \
                   .select("id_model", "price")

# Creo la base de datos en databricks

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS guitar_db")

Out[70]: DataFrame[]

In [0]:
dim_brand.write.mode("overwrite").saveAsTable("guitar_db.dim_brand")
dim_type.write.mode("overwrite").saveAsTable("guitar_db.dim_type")
dim_series.write.mode("overwrite").saveAsTable("guitar_db.dim_series")
dim_model.write.mode("overwrite").saveAsTable("guitar_db.dim_model")
fact_guitar.write.mode("overwrite").saveAsTable("guitar_db.fact_guitar")


In [0]:
%sql
SHOW DATABASES;
SHOW TABLES IN guitar_db;

database,tableName,isTemporary
guitar_db,dim_brand,false
guitar_db,dim_model,false
guitar_db,dim_series,false
guitar_db,dim_type,false
guitar_db,fact_guitar,false


Luego, en otro notebook realizo las consultas en SQL para hacer la comparación de todos los datos